In [1]:
import numpy as np
import os

# Variables globales:

letras = ['A','B','D','E','F','G','H','I','J','K','L','M','N','Ñ','O','P','Q','R','S','T','U','V','W','X','Y','Z']

In [2]:
class Jugador:
    def __init__(self, name, ndim = 10, tipo = 'jugador'):
        self.name = name
        self.tipo = tipo
        self.disparos = []

        tablero = np.full((2,ndim,ndim),'·')
        self.flota, self.radar = tablero[0], tablero[1]
        self.ndim = ndim
        self.letras = letras[:ndim]

    def mostrar_tablero(self):
        ndim = len(self.flota)
        for n in range(2):
            print(end = '  ')
            [print(n, end = ' ') for n in range(1,self.ndim + 1)]
            print(end = '   ')
        print()
        for fila1, fila2, letra in zip(self.flota, self.radar, self.letras):
            print(letra, end = ' ')
            for posicion1 in fila1:
                print(posicion1, end = ' ')
            print(' | ',letra, end = ' ')
            for posicion2 in fila2:
                print(posicion2, end = ' ')
            print()

    def check_barcos(self):
        if np.any(self.flota == 'O'):
            return True
        else:
            return False
    
    def activar_radar(self, flota_enemigo):
        self.radar = np.where(flota_enemigo == 'O', '·', flota_enemigo)
        return self.radar

**Funciones de disparo y limpieza de pantalla**

In [3]:
def recibir_disparo(tablero, coordenada):
    if tablero[coordenada] == "O":
        tablero[coordenada] = "X"
        print("Tocado")
        return True
    elif tablero[coordenada] == "X":
        print("Ya has disparado aquí")
        return False
    else:
        tablero[coordenada] = "~"
        print("Agua")
        return False

def disparo_coordenadas(tablero, coordenada):
    coord = coordenada.split(',')
    if len(coord) != 2:
        return False
    elif coord[0].upper() not in tablero.letras or int(coord[1]) > tablero.ndim:
        return False
    fila = letras.index(coord[0].upper())
    columna = int(coord[1])-1
    return (fila,columna)

def disparo_random(tablero):
    coordenada = (np.random.randint(0,len(tablero)-1),np.random.randint(0,len(tablero)-1))
    while tablero[coordenada] == 'X' or tablero[coordenada] == '~':
        coordenada = (np.random.randint(0,len(tablero)-1),np.random.randint(0,len(tablero)-1))
    recibir_disparo(tablero,coordenada)

def clear(): # Limpiar pantalla
    if os.name == "posix":
        os.system ("clear")
    elif os.name == ("ce", "nt", "dos"):
        os.system ("cls")

**Función de turnos de batalla**

In [13]:
def turno_batalla():
    duo = [jugador,cpu]

    while True:
        atacante, enemigo = duo[0], duo[1]

        print(f'Turno {atacante.name} - CAÑONAZOS: {atacante.disparos.count(True)}/{len(atacante.disparos)}')
        atacante.activar_radar(enemigo.flota)
        atacante.mostrar_tablero()

        if atacante.name == 'vision':
            print(f'Tablero {enemigo.name}')
            enemigo.activar_radar(atacante.flota)
            enemigo.mostrar_tablero()

        if atacante.tipo == 'jugador':
            coordenada = input('Introduce coordenada de disparo: ')
            if coordenada == 'salir':
                break
            disparo = disparo_coordenadas(enemigo, coordenada)
            while disparo == False:
                coordenada = input('Coordenadas incorrectas. Vuelve a intentarlo: ')
                disparo = disparo_coordenadas(enemigo, coordenada)
            acierto = recibir_disparo(enemigo.flota, disparo)
            atacante.disparos.append(acierto)
            if acierto == True:
                continue
        else:
            acierto = disparo_random(enemigo.flota) 
            atacante.disparos.append(acierto)
            if acierto == True:
                continue
        
        clear()    # Limpiamos la pantalla (no funciona en notebooks)
        
        if enemigo.check_barcos() != True:
            print('Ganaste')
            break
        else:
            duo.reverse()
            continue
    
    print('Salida -> Función de menú')

**Prueba:**

In [14]:
jugador = Jugador('Victor')
cpu = Jugador('cpu', tipo = 'cpu')

barcos = [(1,4),(2,4),(3,4),(5,6),(5,7),(5,8)]
disparos = [(1,4), (0,4), (5,4), (5,5), (5,6), (5,7)]
for disparo,barco in zip(disparos,barcos):
    jugador.flota[barco] = 'O'
    cpu.flota[disparo] = 'O'

turno_batalla()

Turno Victor - CAÑONAZOS: 0/0
  1 2 3 4 5 6 7 8 9 10      1 2 3 4 5 6 7 8 9 10    
A · · · · · · · · · ·  |  A · · · · · · · · · · 
B · · · · O · · · · ·  |  B · · · · · · · · · · 
D · · · · O · · · · ·  |  D · · · · · · · · · · 
E · · · · O · · · · ·  |  E · · · · · · · · · · 
F · · · · · · · · · ·  |  F · · · · · · · · · · 
G · · · · · · O O O ·  |  G · · · · · · · · · · 
H · · · · · · · · · ·  |  H · · · · · · · · · · 
I · · · · · · · · · ·  |  I · · · · · · · · · · 
J · · · · · · · · · ·  |  J · · · · · · · · · · 
K · · · · · · · · · ·  |  K · · · · · · · · · · 
Agua
Turno cpu - CAÑONAZOS: 0/0
  1 2 3 4 5 6 7 8 9 10      1 2 3 4 5 6 7 8 9 10    
A · ~ · · O · · · · ·  |  A · · · · · · · · · · 
B · · · · O · · · · ·  |  B · · · · · · · · · · 
D · · · · · · · · · ·  |  D · · · · · · · · · · 
E · · · · · · · · · ·  |  E · · · · · · · · · · 
F · · · · · · · · · ·  |  F · · · · · · · · · · 
G · · · · O O O O · ·  |  G · · · · · · · · · · 
H · · · · · · · · · ·  |  H · · · · · · · · · · 